In [31]:
import pandas as pd

In [32]:
df=pd.read_csv("spotify_millsongdata.csv")
df

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...
...,...,...,...,...
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...


In [33]:
df.shape

(57650, 4)

In [34]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [35]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [36]:
df.head()

,artist,song,text
0,Radiohead,Idioteque,Who's in a bunker? \r\nWho's in a bunker? \r...
1,Selah,Through It All,I've had many tears and sorrows \r\nI've had ...
2,Peter Cetera,The Christmas Song,Chestnuts roasting on an open fire \r\nJack F...
3,Train,Swaying,Don't talk too fast to me-slow-I wanna know \...
4,Enya,We Wish You A Merry Christmas,We wish you a merry Christmas \r\nWe wish you...


In [37]:
df['text'][0]

"Who's in a bunker?  \r\nWho's in a bunker?  \r\nWomen and children first  \r\nAnd the children first  \r\nAnd the children  \r\n  \r\nI'll laugh until my head comes off  \r\nI'll swallow till I burst  \r\nUntil I burst  \r\nUntil I  \r\n  \r\nWho's in a bunker?  \r\nWho's in a bunker?  \r\nI have seen too much  \r\nI haven't seen enough  \r\nYou haven't seen it  \r\n  \r\nI'll laugh until my head comes off  \r\nWomen and children first  \r\nAnd children first  \r\nAnd children  \r\n  \r\nHere I'm alive  \r\nEverything all of the time  \r\nHere I'm alive  \r\nEverything all of the time  \r\n  \r\nIce age coming  \r\nIce age coming  \r\nLet me hear both sides  \r\nLet me hear both sides  \r\nLet me hear both  \r\n  \r\nIce age coming  \r\nIce age coming  \r\nThrow it in the fire  \r\nThrow it in the fire  \r\nThrow it on the  \r\n  \r\nWe're not scaremongering  \r\nThis is really happening  \r\nHappening  \r\nWe're not scaremongering  \r\nThis is really happening  \r\nHappening  \r\n  \

In [38]:
# df=df.sample(5000)

In [39]:
df.shape

(5000, 3)

# Text Preprocessing

In [41]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [42]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [44]:
tfid = TfidfVectorizer(analyzer='word', stop_words = 'english')
matrix = tfid.fit_transform(df['text'])
similar = cosine_similarity(matrix)

In [45]:
similar[0]

array([1.        , 0.00888856, 0.00573547, ..., 0.01350195, 0.00889757,
       0.01084043])

In [50]:
df[df['song']=='Waiting For The Man']
df['song']

0                           Idioteque
1                      Through It All
2                  The Christmas Song
3                             Swaying
4       We Wish You A Merry Christmas
                    ...              
4995                        Fireflies
4996                            Sorry
4997                    Sweet Scarlet
4998           The Power Of The Heart
4999              Faith In Each Other
Name: song, Length: 5000, dtype: object

# Recommendation Function

In [48]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similar[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:5]:
        songs.append(df.iloc[m_id[0]].song)  
    return songs

In [60]:
recommendation("Happy")

['Happy Day',
 "I Just Wanna' Be Happy",
 'Happy Happy Birthday Baby',
 'Stomp Your Hands, Clap Your Feet',
 'Shine All Your Light',
 'Stone Cold',
 'Heart And Head',
 'I Gotta Have A Song',
 'Happiness Comes, Happiness Goes',
 'Beans',
 "You'll Never Walk Alone",
 'Get Happy',
 'Most Anything You Want',
 "When Everybody's Happy",
 'Poor Happy Jimmy',
 'Changes',
 'Trigger Happy',
 'Feel You All Over',
 'I Feel Like A Feather In The Breeze',
 'Shake up Christmas']

In [70]:
import pickle
pickle.dump(similar, open("similarity.pkl","wb"))
pickle.dump(df,open("df.pkl","wb"))